In [ ]:
rawData = pandas.read_csv('Interview.csv')


In [110]:
import pandas
import datetime

def get_cleaned_date(date):
    """
    Return datetime object from a string
    """
    date = str(date).strip()

    if '&' in date:
        date = date.split('&')[0].strip()

    # Since there are a lot of formats in the data, need to handle all the possible options
    date_formats = [
        '%d.%m.%Y', '%d.%m.%y', '%d.%m.%y', '%d-%m-%Y', '%d/%m/%y', '%d/%m/%Y', '%d %b %y', '%d-%b -%y',
        '%d – %b-%y', '%d -%b -%y'
    ]

    for date_format in date_formats:
        try:
            return datetime.datetime.strptime(date, date_format)
        except ValueError:
            pass

#Remove columns that provide no useful data.
cleanData = rawData.drop(['Name(Cand ID)', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27'], axis = 1)

#Remove rows that provide no useful data
cleanData = cleanData.drop(1233)

#Change column names for easier reference
cleanData.columns = [
    'date',
    'client',
    'industry',
    'loc',
    'pos',
    'skillset',
    'interview_type',
    'gender',
    'current_loc',
    'job_loc',
    'venue_loc',
    'native_loc',
    'has_permission',
    'unscheduled',
    'can_call',
    'can_have_alt_number',
    'has_cv_print',
    'venue',
    'call_letter_shared',
    'expected_attendance',
    'observed_attendance',
    'marital_status']

#Fix date formats
cleanData['date'] = cleanData['date'].map(get_cleaned_date)

#We know the date range should be between sep 2014 and jan 2017 so we eliminate all records outside of that range.
cleanData = cleanData[(cleanData['date'] >'2014-09-01') & (cleanData['date'] < '2017-01-31')]


#Make everything lower case and strip whitespace padding for more uniform data
cleanData = pandas.concat([cleanData[c].astype(str).str.lower() for c in cleanData.columns], axis = 1)
cleanData = pandas.concat([cleanData[c].astype(str).str.strip() for c in cleanData.columns], axis = 1)

#Merge standard chartered bank and standard chartered bank chennai. job location is specified in another column.
cleanData['client'].replace('standard chartered bank chennai', 'standard chartered bank', inplace = True)

#aon hewitt, hewitt and aon hewitt gurgaon seem to be the same company, we merge those too
cleanData['client'].replace(['hewitt', 'aon hewitt gurgaon'], 'aon hewitt', inplace = True)

#Merge similar industry items
cleanData['industry'].replace(['it products and services', 'it services'], 'it', inplace = True)

#Fix interview types
cleanData['interview_type'].replace(['scheduled walk in', 'sceduled walkin'], 'scheduled walkin', inplace=True)

#Fix location fields
cleanData['loc'].replace(['- cochin-', 'gurgaonr'], ['cochin', 'gurgaon'], inplace = True)
cleanData['current_loc'].replace('- cochin-', 'cochin', inplace=True)
cleanData['job_loc'].replace('- cochin-', 'cochin', inplace=True)
cleanData['venue_loc'].replace('- cochin-', 'cochin', inplace=True)
cleanData['native_loc'].replace('- cochin-', 'cochin', inplace=True)
cleanData['native_loc'].replace('delhi /ncr', 'delhi', inplace=True)

#We will assume that the time means that the candidate is expected at that time
time_pattern = '\d?\d[.:]\d\d [ap]m'
cleanData['expected_attendance'].replace(time_pattern, 'yes', regex=True, inplace=True)

#Normalize yes/no/na fields
features= ['has_permission', 'unscheduled', 'can_call', 'can_have_alt_number', 'has_cv_print', 'venue', 'call_letter_shared', 'expected_attendance', 'observed_attendance']
no = ['no', 'not yet', 'no dont', 'no- will take it soon', 'no i have only thi number', 'no- i need to check']
na = ['na', 'nan', 'not sure', 'cant say', 'yet to confirm', 'need to check', 'yet to check', 'havent checked', 'uncertain']

for feature in features:
    cleanData[feature].replace(no, -1, inplace=True)
    cleanData[feature].replace(na, 0, inplace=True)
    cleanData[feature].replace('yes', 1, inplace=True)
    
#Normalize gender column
cleanData['gender'].replace('male', 1, inplace=True)
cleanData['gender'].replace('female', -1, inplace=True)


#We drop values that look like dateTime as they make no sense in the skillset column.
filterSeries = cleanData['skillset'].str.contains(time_pattern, regex=True)
cleanData = cleanData[~filterSeries]

#Let's give the field some somblance of an uniform format for easier processing
cleanData['skillset'].replace([
    '/', ' ?, ?', ' developer', 'r & d', 'sccm ?- ?', ' – ra'
], [
    ',', ',', '',  'r&d', 'sccm,',  ''
], inplace=True, regex=True)
cleanData['skillset'].replace([
    'cdd kyc',
    'java j2ee',
    'oracle plsql',
    'core java',
    'senior software engineer-mednet',
    'sr automation testing',
    'tech lead-mednet',
    'ra publishing',
    'java jsf',
    'java,j2ee,core java',
    'java tech lead',
    'automation testing java',
    '- sapbo,informatica',
    'production support - sccm',
    'tech lead- mednet',
    'technical lead',
    'senior analyst',
    'sccm – sharepoint',
    'sccm – sql',
    'tl',
    'sccm,(network,sharepoint,ms exchange)',
    'java-sas',
    'lcm -manager',
    'basesas_program,reporting'
], [
    'cdd,kyc',
    'java,j2ee',
    'plsql',
    'java',
    'senior,developer,mednet',
    'senior,automation,testing',
    'tech lead,mednet',
    'publishing',
    'java,jsf',
    'java,j2ee',
    'java,tech lead',
    'java,automation,testing',
    'sapbo,informatica',
    'sccm,production support',
    'tech lead,mednet',
    'tech lead',
    'senior,analyst',
    'sccm,sharepoint',
    'sccm,sql',
    'tech lead',
    'sccm,network,sharepoint,ms_exchange',
    'java,sas',
    'lcm manager',
    'baseSAS'
    
], inplace=True)
cleanData['skillset'].replace([
    'lending&liablities',
    'l & l',
    'lending & liability'
], 'lending and liabilities', inplace=True)
cleanData['skillset'].replace(['biosimiliars', 'biosimillar'], 'biosimilars', inplace=True)
cleanData['skillset'].replace(' ',  '_', regex=True, inplace=True)

#Lets binarize each item into categories
skillset = cleanData.skillset.str.split(',', expand=True).stack()
dummies = pandas.get_dummies(skillset, prefix='skillset').groupby(level=0).sum()

cleanData = cleanData.join(dummies)

#We drop skillset as it is now encoded
cleanData.drop('skillset', axis=1, inplace=True)

#Encode other categorical features using oneHot
features = ['loc', 'pos', 'client', 'industry', 'interview_type', 'current_loc', 'job_loc', 'venue_loc', 'native_loc', 'marital_status']

for feature in features:
    dummies = pandas.get_dummies(cleanData[feature], prefix=feature)
    cleanData.drop(feature, axis=1, inplace=True)
    cleanData = cleanData.join(dummies)

cleanData.to_excel('excel_clean.xls')